In [1]:
from math import nan
from copy import deepcopy
from datetime import datetime
import itertools
from multiprocessing import Pool
import pickle

In [2]:
import numpy as np
import numpy.random
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay
from catboost import CatBoostClassifier

In [4]:
from load import read_data, read, write, get_full_prepared_data_with_upsample
from processing import start_processing
from parameters import RANDOM_SEED, SCORERS, TARGET_FEATURE, TEST_SIZE, THREADS, TARGET_REPLACER, INVERSE_TARGET_REPLACER
from preparing import (MyOheHotEncoder, MyOrdinalEncoder, MyMinMaxScaler, ColumnsSorter,
                       EmptyColFiller, MyPolynomialFeatures, ordinal_encoding, one_hot_encoding)
from model_preparing import simple_linear, simple_forest, simple_catboost, fast_catboost, gpu_catboost, predict_on_test, ensemble_predict
from tuning import try_each_col, try_wo_each_col, try_cols_in_order

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_info_columns', 1000)

In [6]:
f, ft, fp = get_full_prepared_data_with_upsample()

In [7]:
# raise ValueError

In [8]:
%%time
cols_sets = [None]
result = []
for cols_set in cols_sets:
    ft0 = [(x[0][cols_set], x[1]) for x in ft] if cols_set else ft.copy()
    res = fast_catboost(ft0)
    result.append(res | {'cols': ' '.join(cols_set) if cols_set else nan})
result = pd.DataFrame(result)

Wall time: 1min 28s


In [9]:
result

,train,test_1,test_2,test_3,test_4,cols,model
0,0.873018,0.819016,0.834433,0.837858,0.867402,NaN,(<preparing.MyMinMaxScaler object at 0x000001C...


In [10]:
# _, ax = plt.subplots()
# ax.grid()
# t = pd.DataFrame(model.predict_proba(ft[2]))

# fpr, tpr, _ = roc_curve(ft[3].replace(1, 2).replace(0, 1).replace(2, 0), t[0])
# RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax, c='g')
# print(roc_auc_score(ft[3].replace(1, 2).replace(0, 1).replace(2, 0), t[0]))

# fpr, tpr, _ = roc_curve(ft[3].replace(2, 0), t[1])
# RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax, c='k')
# print(roc_auc_score(ft[3].replace(2, 0), t[1]))

# fpr, tpr, _ = roc_curve(ft[3].replace(1, 0).replace(2, 1), t[2])
# RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax, c='r')
# print(roc_auc_score(ft[3].replace(1, 0).replace(2, 1), t[2]))

In [11]:
%%time
t0 = predict_on_test(model, ft, fp, refit=False)

predict: 0.8716686756737314


,Статус,status
0.0,0.689583,0.607259
1.0,0.255418,0.347541
2.0,0.054999,0.045200


Wall time: 239 ms
